# NFL Feature Store

1. Config
2. Find data sources
3. Collect data sources
4. Create features
5. Vizualize
6. Store Features

## 1. Config

In [14]:
import pandas as pd
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)


import requests



## 2. Find data sources

NFLDATAVERSE: https://github.com/nflverse



In [4]:
NFLVERSEGITHUB = "https://github.com/nflverse/nflverse-data/releases/download/"
NFL_PLAYER_URL = f"{NFLVERSEGITHUB}players/players.parquet"
NFL_PLAYER_STATS_URL = f"{NFLVERSEGITHUB}player_stats/player_stats.parquet"
NFL_PBP = NFLVERSEGITHUB + "pbp/play_by_play_{season}.parquet"

## 3. Collect data sources

In [228]:
player_df = pd.read_parquet(NFL_PLAYER_URL)
player_stats_df = pd.read_parquet(NFL_PLAYER_STATS_URL)
player_stat_columns = [
    'player_id', 
    #'player_name', 
    #'player_display_name', 
    #'position', 
    #'position_group', 
    #'headshot_url', 
    #'recent_team', 
    'season', 
    #'week', 
    #'season_type', 
    #'opponent_team', 
    'completions', 
    'attempts', 
    'passing_yards', 
    'passing_tds', 
    'interceptions', 
    'sacks', 
    'sack_yards', 
    'sack_fumbles', 
    'sack_fumbles_lost', 
    'passing_air_yards', 
    'passing_yards_after_catch', 
    'passing_first_downs', 
    'passing_epa', 
    'passing_2pt_conversions', 
    'pacr', 
    'dakota', 
    'carries', 
    'rushing_yards', 
    'rushing_tds', 
    'rushing_fumbles', 
    'rushing_fumbles_lost', 
    'rushing_first_downs', 
    'rushing_epa', 
    'rushing_2pt_conversions', 
    'receptions', 
    'targets', 
    'receiving_yards', 
    'receiving_tds', 
    'receiving_fumbles', 
    'receiving_fumbles_lost', 
    'receiving_air_yards', 
    'receiving_yards_after_catch', 
    'receiving_first_downs', 
    'receiving_epa', 
    'receiving_2pt_conversions', 
    'racr', 
    'target_share', 
    'air_yards_share', 
    'wopr', 
    'special_teams_tds', 
    'fantasy_points', 
    #'fantasy_points_ppr'
]
'''
player_stat_columns = [
    'player_id', 
    'season', 
    'fantasy_points', 
    #'fantasy_points_ppr'

]
'''

player_points_columns = [
    'player_id', 
    'season', 
    'fantasy_points', 
    #'fantasy_points_ppr'

]
#pbp_df = pd.read_parquet(NFL_PBP.format(season=2023))

a = player_stats_df[player_stats_df.season_type == 'REG'].copy()
player_stats_df = a[player_stat_columns].copy()
player_points_df = a[player_points_columns].groupby(['player_id', 'season'])['fantasy_points'].sum().reset_index()

In [229]:
# Add a column for the number of games played by each player in each season
player_id = 'player_id'

player_stats_df['games_played'] = 1

# Group by player and season to get total stats
total_stats = player_stats_df.groupby([player_id, 'season']).sum().reset_index()

total_stats = total_stats[(total_stats.games_played > 3)].copy()

# Calculate average stats per game
avg_stats = total_stats.copy()
stat_columns = total_stats.columns.difference([player_id, 'season', 'games_played'])
avg_stats[stat_columns] = avg_stats[stat_columns].div(avg_stats['games_played'], axis=0)

# Drop games_played column from avg_stats
avg_stats.drop(columns=['games_played'], inplace=True)

# Player table
player_attrs = ['display_name', 'position', 'height', 'weight','entry_year','birth_date','draft_number','college_name']
player_info_df = player_df[player_df.position.isin(['QB', 'RB', 'WR', 'TE'])][['gsis_id']+player_attrs].copy()
player_info_df.rename(columns={'gsis_id': 'player_id'}, inplace=True)

## Generate n last year stats for each player and join them to the player_info_df

for n_last in [0,1,2]:
    t_copy = total_stats.copy()
    avg_copy = avg_stats.copy()
    prefix ='last_year_' if n_last == 0 else f'{n_last+1}_years_ago_'

    t_copy['season'] = t_copy['season'] + n_last
    t_copy.columns = [f'total_{prefix}{col}' if col not in ['player_id', 'season'] else col for col in t_copy.columns]
    join_cols = ['player_id', 'season'] if 'season' in player_info_df.columns else ['player_id']
    player_info_df = pd.merge(player_info_df, t_copy, on=join_cols, how='left')

    avg_copy['season'] = avg_copy['season'] + n_last
    avg_copy.columns = [f'avg_{prefix}{col}' if col not in ['player_id', 'season'] else col for col in avg_copy.columns]
    player_info_df = pd.merge(player_info_df, avg_copy, on=['player_id', 'season'], how='left')

# Fillna and adjust to projection season
player_info_df.fillna(-1, inplace=True)
player_info_df = player_info_df[player_info_df.season != -1].copy()
player_info_df['season'] = player_info_df['season'] + 1
player_info_df['season'] = player_info_df['season'].astype(int)

player_info_df = player_info_df.drop(columns=player_attrs)

a = player_info_df.sort_values(by='season')
y = player_points_df.sort_values(by='season')
feature_store_df =pd.merge(a, y,on=['player_id', 'season'], how='left')

player_info_df = player_df[player_df.position.isin(['QB', 'RB', 'WR', 'TE'])][['gsis_id']+player_attrs].copy()
player_info_df.rename(columns={'gsis_id': 'player_id'}, inplace=True)
feature_store_df = pd.merge(feature_store_df, player_info_df, on=['player_id'], how='left')

feature_store_df['years_of_experience'] = feature_store_df['season'] - feature_store_df['entry_year']
feature_store_df.fillna(-1, inplace=True)

# rank by position and season for ranked by total fantasy points and make a column called position_rank

feature_store_df = feature_store_df[(feature_store_df.fantasy_points > 0)].copy()

feature_store_df['position_rank'] = feature_store_df.groupby(['position', 'season'])['fantasy_points'].rank(ascending=False, method='first')

feature_store_df[(feature_store_df.fantasy_points <= 0)].copy()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023])

,player_id,season,total_last_year_completions,total_last_year_attempts,total_last_year_passing_yards,total_last_year_passing_tds,total_last_year_interceptions,total_last_year_sacks,total_last_year_sack_yards,total_last_year_sack_fumbles,total_last_year_sack_fumbles_lost,total_last_year_passing_air_yards,total_last_year_passing_yards_after_catch,total_last_year_passing_first_downs,total_last_year_passing_epa,total_last_year_passing_2pt_conversions,total_last_year_pacr,total_last_year_dakota,total_last_year_carries,total_last_year_rushing_yards,total_last_year_rushing_tds,total_last_year_rushing_fumbles,total_last_year_rushing_fumbles_lost,total_last_year_rushing_first_downs,total_last_year_rushing_epa,total_last_year_rushing_2pt_conversions,total_last_year_receptions,total_last_year_targets,total_last_year_receiving_yards,total_last_year_receiving_tds,total_last_year_receiving_fumbles,total_last_year_receiving_fumbles_lost,total_last_year_receiving_air_yards,total_last_year_receiving_yards_after_catch,total_last_year_receiving_first_downs,total_last_year_receiving_epa,total_last_year_receiving_2pt_conversions,total_last_year_racr,total_last_year_target_share,total_last_year_air_yards_share,total_last_year_wopr,total_last_year_special_teams_tds,total_last_year_fantasy_points,total_last_year_games_played,avg_last_year_completions,avg_last_year_attempts,avg_last_year_passing_yards,avg_last_year_passing_tds,avg_last_year_interceptions,avg_last_year_sacks,avg_last_year_sack_yards,avg_last_year_sack_fumbles,avg_last_year_sack_fumbles_lost,avg_last_year_passing_air_yards,avg_last_year_passing_yards_after_catch,avg_last_year_passing_first_downs,avg_last_year_passing_epa,avg_last_year_passing_2pt_conversions,avg_last_year_pacr,avg_last_year_dakota,avg_last_year_carries,avg_last_year_rushing_yards,avg_last_year_rushing_tds,avg_last_year_rushing_fumbles,avg_last_year_rushing_fumbles_lost,avg_last_year_rushing_first_downs,avg_last_year_rushing_epa,avg_last_year_rushing_2pt_conversions,avg_last_year_receptions,avg_last_year_targets,avg_last_year_receiving_yards,avg_last_year_receiving_tds,avg_last_year_receiving_fumbles,avg_last_year_receiving_fumbles_lost,avg_last_year_receiving_air_yards,avg_last_year_receiving_yards_after_catch,avg_last_year_receiving_first_downs,avg_last_year_receiving_epa,avg_last_year_receiving_2pt_conversions,avg_last_year_racr,avg_last_year_target_share,avg_last_year_air_yards_share,avg_last_year_wopr,avg_last_year_special_teams_tds,avg_last_year_fantasy_points,total_2_years_ago_completions,total_2_years_ago_attempts,total_2_years_ago_passing_yards,total_2_years_ago_passing_tds,total_2_years_ago_interceptions,total_2_years_ago_sacks,total_2_years_ago_sack_yards,total_2_years_ago_sack_fumbles,total_2_years_ago_sack_fumbles_lost,total_2_years_ago_passing_air_yards,total_2_years_ago_passing_yards_after_catch,total_2_years_ago_passing_first_downs,total_2_years_ago_passing_epa,total_2_years_ago_passing_2pt_conversions,total_2_years_ago_pacr,...,avg_2_years_ago_receiving_2pt_conversions,avg_2_years_ago_racr,avg_2_years_ago_target_share,avg_2_years_ago_air_yards_share,avg_2_years_ago_wopr,avg_2_years_ago_special_teams_tds,avg_2_years_ago_fantasy_points,total_3_years_ago_completions,total_3_years_ago_attempts,total_3_years_ago_passing_yards,total_3_years_ago_passing_tds,total_3_years_ago_interceptions,total_3_years_ago_sacks,total_3_years_ago_sack_yards,total_3_years_ago_sack_fumbles,total_3_years_ago_sack_fumbles_lost,total_3_years_ago_passing_air_yards,total_3_years_ago_passing_yards_after_catch,total_3_years_ago_passing_first_downs,total_3_years_ago_passing_epa,total_3_years_ago_passing_2pt_conversions,total_3_years_ago_pacr,total_3_years_ago_dakota,total_3_years_ago_carries,total_3_years_ago_rushing_yards,total_3_years_ago_rushing_tds,total_3_years_ago_rushing_fumbles,total_3_years_ago_rushing_fumbles_lost,total_3_years_ago_rushing_first_downs,total_3_years_ago_rushing_epa,total_3_years_ago_rushing_2pt_conversions,total

In [221]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

def train_fantasy_model(feature_store_df, position, attributes, year):
    train_years = list(range(2010, year-1))
    holdout_year = year - 1
    test_year = year
    drop_cols = ['player_id', 'season', 'fantasy_points', 'display_name', 'college_name', 'birth_date', 'entry_year', 'position']

    # Filter for the specific position
    pos_df = feature_store_df[feature_store_df['position'] == position].copy()

    if position == 'QB':
        filt = (pos_df['total_last_year_completions'] >= 0) | (pos_df['total_2_years_ago_completions'] >= 0) | (pos_df['years_of_experience'] == 0)
        print(pos_df.shape[0]-sum(filt))
        pos_df = pos_df[filt].copy()

    # Split the data
    train_df = pos_df[pos_df['season'].isin(train_years)].copy()
    holdout_df = pos_df[pos_df['season'] == holdout_year].copy()
    test_df = pos_df[pos_df['season'] == test_year].copy()

    # Define features and target variable
    X_train = train_df[attributes]
    y_train = train_df['fantasy_points']

    X_holdout = holdout_df[attributes]
    y_holdout = holdout_df['fantasy_points']

    X_test = test_df[attributes]
    y_test = test_df['fantasy_points']

    # Train a Random Forest model
    model = RandomForestRegressor(n_estimators=400, random_state=42)
    model.fit(X_train, y_train)

    # Predict and evaluate
    holdout_predictions = model.predict(X_holdout)
    test_predictions = model.predict(X_test)
    holdout_mae = mean_absolute_error(y_holdout, holdout_predictions)
    test_mae = mean_absolute_error(y_test, test_predictions)
    print(f'Mean Absolute Error for {position}s on holdout set: {holdout_mae}')
    print(f'Mean Absolute Error for {position}s on test set: {test_mae}')

    # Feature importances
    feature_importances = model.feature_importances_
    feature_names = X_train.columns
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importances
    }).sort_values(by='importance', ascending=False)
    test_df = test_df[drop_cols]
    test_df['predicted_fantasy_points'] = test_predictions

    return test_df, feature_importance_df

# Parameters

# Train and evaluate model for each position
positions = ['QB']

QB_MODEL_ATTRIBUTES = ['total_last_year_fantasy_points',
 'avg_last_year_fantasy_points',
 'total_last_year_passing_tds',
 'draft_number',
 'total_2_years_ago_passing_yards',
 'total_2_years_ago_dakota',
 'total_last_year_games_played',
 'total_last_year_passing_yards_after_catch',
 'total_last_year_dakota',
 'total_last_year_carries',
 'avg_last_year_sack_yards',
 'weight',
 'avg_last_year_passing_tds',
 'avg_3_years_ago_passing_air_yards',
 'avg_last_year_passing_yards_after_catch',
 'avg_last_year_passing_air_yards',
 'total_last_year_passing_air_yards',
 'avg_last_year_attempts',
 'total_2_years_ago_passing_epa',
 'total_last_year_sacks',
 'avg_last_year_sacks',
 'avg_last_year_passing_epa',
 'total_last_year_passing_yards',
 'avg_last_year_passing_yards',
 'total_2_years_ago_rushing_epa',
 'total_2_years_ago_completions',
 'total_last_year_passing_epa',
 'total_2_years_ago_games_played',
 'total_last_year_attempts',
 'avg_2_years_ago_rushing_epa',
 'total_last_year_rushing_epa',
 'avg_last_year_interceptions',
 'total_2_years_ago_passing_first_downs',
 'avg_last_year_rushing_epa',
 'total_last_year_rushing_yards',
 'avg_last_year_rushing_fumbles',
 'avg_last_year_dakota',
 'avg_2_years_ago_passing_epa',
 'avg_last_year_sack_fumbles',
 'total_last_year_completions',
 'avg_last_year_rushing_yards',
 'total_last_year_rushing_first_downs',
 'total_last_year_interceptions',
 'total_last_year_sack_yards',
 'avg_last_year_passing_first_downs']


year = 2023
for position in positions:
    test_df,feature_importance_df = train_fantasy_model(
        feature_store_df, position, QB_MODEL_ATTRIBUTES, year
    )

    print(f'Feature importances for {position}s:')
    print(feature_importance_df.head(10))

0
Mean Absolute Error for QBs on holdout set: 65.02781764705882
Mean Absolute Error for QBs on test set: 72.86178673469385
Feature importances for QBs:
                              feature  importance
0      total_last_year_fantasy_points    0.371527
1        avg_last_year_fantasy_points    0.112790
2         total_last_year_passing_tds    0.055616
3                        draft_number    0.022154
4     total_2_years_ago_passing_yards    0.019322
5            total_2_years_ago_dakota    0.016476
9             total_last_year_carries    0.016183
15    avg_last_year_passing_air_yards    0.015196
13  avg_3_years_ago_passing_air_yards    0.014680
12          avg_last_year_passing_tds    0.014253


In [222]:
test_df

,player_id,season,fantasy_points,display_name,college_name,birth_date,entry_year,position,predicted_fantasy_points
9124,00-0034796,2023,331.22,Lamar Jackson,Louisville,1997-01-07,2018.0,QB,233.75230
9129,00-0035993,2023,25.62,Tyler Huntley,Utah,1998-02-03,2020.0,QB,74.03855
9162,00-0026498,2023,243.10,Matthew Stafford,Georgia,1988-02-07,2009.0,QB,192.88475
9172,00-0036264,2023,319.06,Jordan Love,Utah State,1998-11-02,2020.0,QB,18.01585
9181,00-0030565,2023,226.26,Geno Smith,West Virginia,1990-10-10,2013.0,QB,277.03805
9201,00-0037013,2023,119.94,Zach Wilson,Brigham Young,-1,2021.0,QB,116.21205
9202,00-0034857,2023,394.64,Josh Allen,Wyoming,1996-05-21,2018.0,QB,292.16280
9206,00-0034855,2023,274.06,Baker Mayfield,Oklahoma,1995-04-14,2018.0,QB,126.61055
9208,00-0036212,2023,272.36,Tua Tagovailoa,Alabama,1998-03-02,2020.0,QB,245.98980
9228,00-0036971,2023,262.54,Trevor Lawrence,Clemson,-1,2021.0,QB,251.57010


In [218]:
list(feature_importance_df.feature.values[0:45])

['total_last_year_fantasy_points',
 'avg_last_year_fantasy_points',
 'total_last_year_passing_tds',
 'draft_number',
 'total_2_years_ago_passing_yards',
 'total_2_years_ago_dakota',
 'total_last_year_games_played',
 'total_last_year_passing_yards_after_catch',
 'total_last_year_dakota',
 'total_last_year_carries',
 'avg_last_year_sack_yards',
 'weight',
 'avg_last_year_passing_tds',
 'avg_3_years_ago_passing_air_yards',
 'avg_last_year_passing_yards_after_catch',
 'avg_last_year_passing_air_yards',
 'total_last_year_passing_air_yards',
 'avg_last_year_attempts',
 'total_2_years_ago_passing_epa',
 'total_last_year_sacks',
 'avg_last_year_sacks',
 'avg_last_year_passing_epa',
 'total_last_year_passing_yards',
 'avg_last_year_passing_yards',
 'total_2_years_ago_rushing_epa',
 'total_2_years_ago_completions',
 'total_last_year_passing_epa',
 'total_2_years_ago_games_played',
 'total_last_year_attempts',
 'avg_2_years_ago_rushing_epa',
 'total_last_year_rushing_epa',
 'avg_last_year_interce

In [32]:
player_df = pd.read_csv('https://github.com/dynastyprocess/data/raw/master/files/db_playerids.csv')
player_df

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,NaN,NaN,11560.0,NaN,4431611.0,40900.0,NaN,2669577.0,NaN,NaN,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,QB,CHI,2001-11-18,22.6,2024.0,1.0,1.0,NaN,NaN,73.0,215.0,USC,2024
1,16580,d0c0630e-8925-4b81-a32b-e4146d95f01f,23046.0,NaN,NaN,11564.0,NaN,4431452.0,40881.0,NaN,2669575.0,NaN,NaN,17673.0,NaN,1551.0,40881.0,0.0,NaN,1286803.0,Drake Maye,drake maye,QB,NEP,2002-08-30,21.9,2024.0,1.0,3.0,NaN,NaN,76.0,225.0,North Carolina,2024
2,16581,bab8c65a-9fff-410d-8e34-dc2f636d890e,22902.0,NaN,NaN,11566.0,NaN,4426348.0,40896.0,NaN,3123337.0,NaN,NaN,17692.0,NaN,1552.0,40896.0,0.0,NaN,1161228.0,Jayden Daniels,jayden daniels,QB,WAS,2000-12-18,23.6,2024.0,1.0,2.0,NaN,NaN,76.0,210.0,LSU,2024
3,16582,f056c8ed-6ee5-492c-af11-3f7e3d97195c,22910.0,NaN,NaN,11563.0,NaN,4426338.0,40875.0,NaN,3121193.0,NaN,NaN,16995.0,NaN,1553.0,40875.0,0.0,NaN,1161899.0,Bo Nix,bo nix,QB,DEN,2000-02-25,24.4,2024.0,1.0,12.0,NaN,NaN,74.0,217.0,Oregon,2024
4,16583,6499a161-b221-4591-b122-d99ece1be64b,22973.0,NaN,NaN,11559.0,NaN,4360423.0,40889.0,NaN,2963496.0,NaN,NaN,17700.0,NaN,1554.0,40889.0,0.0,NaN,1107478.0,Michael Penix Jr.,michael penix,QB,ATL,2000-05-08,24.2,2024.0,1.0,8.0,NaN,NaN,74.0,216.0,Washington,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11680,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11681,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11682,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11683,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bubby Brister,bubby brister,QB,KCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024


In [34]:
fp_df.page_type.unique()

fp_df[fp_df.page_type=='best-overall']

array(['best-overall', 'best-dst', 'best-qb', 'best-rb', 'best-te',
       'best-wr', 'dynasty-offense', 'weekly-offense', 'dynasty-idp',
       'dynasty-overall', 'weekly-kdst', 'dynasty-db', 'dynasty-dl',
       'dynasty-dst', 'dynasty-k', 'dynasty-lb', 'dynasty-qb',
       'dynasty-rb', 'dynasty-te', 'dynasty-wr', 'dynasty-rk',
       'dynasty-op', 'redraft-offense', 'redraft-idp', 'redraft-overall',
       'redraft-kdst', 'redraft-db', 'redraft-dl', 'redraft-dst',
       'redraft-k', 'redraft-lb', 'redraft-qb', 'redraft-rb',
       'redraft-te', 'redraft-wr', 'redraft-op', 'weekly-idp',
       'weekly-db', 'weekly-dl', 'weekly-dst', 'weekly-k', 'weekly-lb',
       'weekly-qb', 'weekly-rb', 'weekly-te', 'weekly-wr', 'weekly-op'],
      dtype=object)

In [36]:
fp_df[fp_df.page_type=='weekly-db']

,fp_page,page_type,player,id,pos,team,ecr,sd,best,worst,mergename,tm,sportsdata_id,player_filename,yahoo_id,cbs_id,player_owned_avg,player_owned_espn,player_owned_yahoo,player_image_url,player_square_image_url,rank_delta,ecr_type,scrape_date
1146193,/nfl/rankings/db.php,weekly-db,Budda Baker,16733,DB,ARI,1.00,0.00,1.0,1.0,budda baker,ARI,5ce20f3e-0f02-4f53-a2ef-5b076361f2b1,budda-baker.php,30149,2139625,64.8,65.5,64.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2020-10-16
1146194,/nfl/rankings/db.php,weekly-db,Landon Collins,13895,DB,WAS,2.67,0.75,2.0,10.0,landon collins,WAS,a9c41c5b-0dcf-40cc-a76c-644307f2f2df,landon-collins.php,28421,2000901,40.1,52.1,28.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2020-10-16
1146195,/nfl/rankings/db.php,weekly-db,Jeremy Chinn,19429,DB,CAR,4.00,1.15,3.0,11.0,jeremy chinn,CAR,5f623fbc-415e-4035-b423-7850cf1153b4,jeremy-chinn.php,32734,2247190,22.0,25.9,18.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,1.0,wp,2020-10-16
1146196,/nfl/rankings/db.php,weekly-db,Jordan Poyer,11759,DB,BUF,4.33,0.94,2.0,9.0,jordan poyer,BUF,95fab6fa-3ee1-47d0-93ad-c7ff41744be7,jordan-poyer.php,26841,1665374,49.5,47.9,51.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,-1.0,wp,2020-10-16
1146197,/nfl/rankings/db.php,weekly-db,John Johnson,16747,DB,LAR,5.83,1.95,2.0,15.0,john johnson,LAR,8c824157-eb33-4378-bf19-6c738a186ceb,john-johnson.php,30204,2082526,52.5,53.0,52.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2020-10-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197129,/nfl/rankings/db.php,weekly-db,K'Von Wallace,19319,DB,TEN,97.00,9.00,88.0,106.0,KVon Wallace,TEN,789af1aa-253e-4fda-a93b-cef346bd91b3,kvon-wallace.php,32797,2239538,1.7,2.3,1.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,-55.0,wp,2023-12-29
1197130,/nfl/rankings/db.php,weekly-db,Sean Murphy-Bunting,18648,DB,TEN,106.00,15.00,91.0,121.0,Sean Murphy-Bunting,TEN,f84bf885-d6ff-4fc8-8b6e-64bb3bceb17d,sean-bunting.php,31871,2188984,1.9,1.9,0.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,-17.0,wp,2023-12-29
1197131,/nfl/rankings/db.php,weekly-db,Jaylon Johnson,19254,DB,CHI,110.00,17.00,93.0,127.0,Jaylon Johnson,CHI,99b81b41-fb3b-4650-940b-9cb3770021ba,jaylon-johnson.php,32720,2827532,0.7,0.7,0.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2023-12-29
1197132,/nfl/rankings/db.php,weekly-db,Ronnie Harrison Jr.,17319,DB,IND,122.00,26.00,96.0,148.0,Ronnie Harrison,IND,9b96ef63-04bd-41ae-b59c-acc0f2561d19,ronnie-harrison.php,31063,2180568,0.2,0.2,0.0,https://images.fantasypros.com/images/players/...,https://images.fantasypros.com/images/players/...,NaN,wp,2023-12-29
